In [1]:
pip install vaderSentiment


  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 320.0 kB/s eta 0:00:01
   ------------------- ------------------- 61.4/126.0 kB 465.5 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 820.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
lyrics = [
    "When I find myself in times of trouble, Mother Mary comes to me, speaking words of wisdom, let it be.",
    "I can't get no satisfaction, I can't get no satisfaction, 'cause I try and I try and I try and I try, I can't get no.",
    "Hey Jude, don't make it bad. Take a sad song and make it better."
]


In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment of lyrics
def analyze_lyrics(lyrics):
    results = []
    for lyric in lyrics:
        scores = analyzer.polarity_scores(lyric)
        results.append({
            'lyric': lyric,
            'compound': scores['compound'],
            'positive': scores['pos'],
            'neutral': scores['neu'],
            'negative': scores['neg']
        })
    return results

# Perform the analysis
lyric_sentiments = analyze_lyrics(lyrics)

# Output the results
for sentiment in lyric_sentiments:
    print(f"Lyrics: {sentiment['lyric']}")
    print(f"Compound Score: {sentiment['compound']}")
    print(f"Positive Score: {sentiment['positive']}")
    print(f"Neutral Score: {sentiment['neutral']}")
    print(f"Negative Score: {sentiment['negative']}")
    print()


Lyrics: When I find myself in times of trouble, Mother Mary comes to me, speaking words of wisdom, let it be.
Compound Score: 0.1779
Positive Score: 0.141
Neutral Score: 0.747
Negative Score: 0.112

Lyrics: I can't get no satisfaction, I can't get no satisfaction, 'cause I try and I try and I try and I try, I can't get no.
Compound Score: 0.4733
Positive Score: 0.145
Neutral Score: 0.855
Negative Score: 0.0

Lyrics: Hey Jude, don't make it bad. Take a sad song and make it better.
Compound Score: 0.3919
Positive Score: 0.29
Neutral Score: 0.554
Negative Score: 0.156



In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Add custom words to the lexicon if needed
analyzer.lexicon.update({
    "can't get no": -2.0  # Custom entry for specific phrase
})

# Sample lyrics
lyrics = "I can't get no satisfaction, I can't get no satisfaction, 'cause I try and I try and I try and I try, I can't get no."

# Perform sentiment analysis
scores = analyzer.polarity_scores(lyrics)

# Output the sentiment scores
print(f"Lyrics: {lyrics}")
print(f"Compound Score: {scores['compound']}")
print(f"Positive Score: {scores['pos']}")
print(f"Neutral Score: {scores['neu']}")
print(f"Negative Score: {scores['neg']}")


Lyrics: I can't get no satisfaction, I can't get no satisfaction, 'cause I try and I try and I try and I try, I can't get no.
Compound Score: 0.4733
Positive Score: 0.145
Neutral Score: 0.855
Negative Score: 0.0


In [5]:
pip install pandas vaderSentiment


Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

# Load the dataset
df = pd.read_csv('selected_lyrics.csv')


In [12]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the dataset
df = pd.read_csv('selected_lyrics.csv')

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment
def analyze_sentiment(lyric):
    if pd.isna(lyric):
        # Return neutral sentiment for NaN values
        return pd.Series([0, 0, 1, 0])
    else:
        scores = analyzer.polarity_scores(str(lyric))  # Ensure lyric is a string
        return pd.Series([scores['compound'], scores['pos'], scores['neu'], scores['neg']])

# Apply the function to the Cleared_Lyrics column and create new columns for the sentiment scores
df[['compound', 'positive', 'neutral', 'negative']] = df['Cleaned_Lyrics'].apply(analyze_sentiment)

# Display the DataFrame with sentiment scores
print(df.head())

# Save the resulting DataFrame to a new CSV file
df.to_csv('lyrics_sentiment_analysis.csv', index=False)



         Artist             Title  Genre  \
0  3 doors down  here without you  Metal   
1  3 doors down     when i'm gone  Metal   
2  3 doors down        kryptonite  Metal   
3  3 doors down         let me go  Metal   
4  3 doors down      be like that  Metal   

                                      Cleaned_Lyrics  compound  positive  \
0  A hundred days have made me older\nSince the l...    0.9797     0.134   
1  Theres another world inside of me that you may...   -0.3661     0.132   
2  I took a walk around the world\nTo ease my tro...    0.8625     0.146   
3  One more kiss could be the best thing\none mor...    0.9934     0.186   
4  He spends his nights in California\nWatching t...    0.9923     0.161   

   neutral  negative  
0    0.806     0.060  
1    0.709     0.159  
2    0.751     0.102  
3    0.728     0.086  
4    0.818     0.021  


In [13]:
# Find the instance with the highest negative score
max_negative_index = df['negative'].idxmax()
max_negative_instance = df.loc[max_negative_index]

# Display the instance with the highest negative score
print("Instance with the highest negative score:")
print(max_negative_instance)

Instance with the highest negative score:
Artist            blind guardian
Title                    lammoth
Genre                      Metal
Cleaned_Lyrics        War Sounds
compound                 -0.5994
positive                     0.0
neutral                    0.204
negative                   0.796
Name: 95864, dtype: object


In [16]:
# Function to categorize sentiment based on scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
df['sentiment_category'] = df.apply(categorize_sentiment, axis=1)

# Group by genre and sentiment category, then count the occurrences
sentiment_counts = df.groupby(['Genre', 'sentiment_category']).size().unstack(fill_value=0)

# Display the sentiment counts by genre
print(sentiment_counts)

sentiment_category  negative  neutral  positive
Genre                                          
Jazz                    2160      163     10991
Metal                  12033      240      6860
Pop                    26766      779     58899
Rap                     7663       69      5147


In [18]:
# Function to categorize sentiment based on scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
df['sentiment_category'] = df.apply(categorize_sentiment, axis=1)

# Define the number of instances to sample per genre
num_samples = 1200

# Sample 1200 instances from each genre
genres = ['Metal', 'Pop', 'Jazz', 'Rap']
sampled_dfs = [df[df['Genre'] == genre].sample(n=num_samples, random_state=42) for genre in genres]

# Concatenate the sampled DataFrames
sampled_df = pd.concat(sampled_dfs)

# Save the sampled DataFrame to a new CSV file
sampled_df.to_csv('balanced_lyrics_dataset.csv', index=False)

In [19]:
import pandas as pd

# Load the balanced dataset
balanced_df = pd.read_csv('balanced_lyrics_dataset.csv')

# Function to categorize sentiment based on compound scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
balanced_df['sentiment_category'] = balanced_df.apply(categorize_sentiment, axis=1)

# Group by genre and sentiment category, then count the occurrences
sentiment_counts = balanced_df.groupby(['Genre', 'sentiment_category']).size().unstack(fill_value=0)

# Display the sentiment counts by genre
print(sentiment_counts)


sentiment_category  negative  neutral  positive
Genre                                          
Jazz                     189       14       997
Metal                    775        9       416
Pop                      343       16       841
Rap                      706        6       488


In [20]:
import pandas as pd

# Load the balanced dataset
balanced_df = pd.read_csv('nltk_preprocessed_lyrics_dataset.csv')

# Function to categorize sentiment based on compound scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
balanced_df['sentiment_category'] = balanced_df.apply(categorize_sentiment, axis=1)

# Group by genre and sentiment category, then count the occurrences
sentiment_counts = balanced_df.groupby(['Genre', 'sentiment_category']).size().unstack(fill_value=0)

# Display the sentiment counts by genre
print(sentiment_counts)

sentiment_category  negative  neutral  positive
Genre                                          
Jazz                     189       14       997
Metal                    775        9       416
Pop                      343       16       841
Rap                      706        6       488


In [1]:
import pandas as pd

# Load the balanced dataset
balanced_df = pd.read_csv('preprocessed_lyrics_dataset.csv')

# Function to categorize sentiment based on compound scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
balanced_df['sentiment_category'] = balanced_df.apply(categorize_sentiment, axis=1)

# Group by genre and sentiment category, then count the occurrences
sentiment_counts = balanced_df.groupby(['Genre', 'sentiment_category']).size().unstack(fill_value=0)

# Display the sentiment counts by genre
print(sentiment_counts)

sentiment_category  negative  neutral  positive
Genre                                          
Jazz                     189       14       997
Metal                    775        9       416
Pop                      343       16       841
Rap                      706        6       488


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv('1500date-mai multe genuri.csv')


In [3]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the dataset
df = pd.read_csv('1500date-mai multe genuri.csv')

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment
def analyze_sentiment(lyric):
    if pd.isna(lyric):
        # Return neutral sentiment for NaN values
        return pd.Series([0, 0, 1, 0])
    else:
        scores = analyzer.polarity_scores(str(lyric))  # Ensure lyric is a string
        return pd.Series([scores['compound'], scores['pos'], scores['neu'], scores['neg']])

# Apply the function to the Cleared_Lyrics column and create new columns for the sentiment scores
df[['compound', 'positive', 'neutral', 'negative']] = df['Cleaned_Lyrics'].apply(analyze_sentiment)

# Display the DataFrame with sentiment scores
print(df.head())

# Save the resulting DataFrame to a new CSV file
df.to_csv('1500_sentiment_analysis.csv', index=False)

      Artist                    Title Genre  \
0  12 stones            world so cold  Rock   
1  12 stones                   broken  Rock   
2  12 stones             3 leaf loser  Rock   
3  12 stones  anthem for the underdog  Rock   
4  12 stones               adrenaline  Rock   

                                      Cleaned_Lyrics  compound  positive  \
0  It starts with pain followed by hate\nFueled b...   -0.9813     0.107   
1  Freedom\nAlone again again alone\nPatiently wa...   -0.9973     0.103   
2  Biting the hand that feeds you lying to the vo...   -0.9539     0.066   
3  You say you know just who I am\nBut you cant i...    0.3086     0.165   
4  My heart is beating faster cant control these ...    0.9953     0.213   

   neutral  negative  
0    0.716     0.177  
1    0.579     0.318  
2    0.770     0.164  
3    0.697     0.139  
4    0.710     0.077  


In [4]:
import pandas as pd

# Load the balanced dataset
balanced_df = pd.read_csv('1500_sentiment_analysis.csv')

# Function to categorize sentiment based on compound scores
def categorize_sentiment(row):
    if row['compound'] >= 0.05:
        return 'positive'
    elif row['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the categorize_sentiment function to create a new column for sentiment category
balanced_df['sentiment_category'] = balanced_df.apply(categorize_sentiment, axis=1)

# Group by genre and sentiment category, then count the occurrences
sentiment_counts = balanced_df.groupby(['Genre', 'sentiment_category']).size().unstack(fill_value=0)

# Display the sentiment counts by genre
print(sentiment_counts)


sentiment_category  negative  neutral  positive
Genre                                          
Country                  346       25      1129
Electronic               523       44       933
Folk                     397       22      1081
Hip-Hop                  798        8       694
Indie                    501       36       963
Jazz                     252       22      1226
Metal                    853       13       634
Pop                      664       22       814
R&B                      230       10      1260
Rock                     598       10       892
rap                      855        8       637
